In [8]:
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
import os
import re
import csv
import requests
import urllib.request
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time


In [2]:
import requests

url = "https://open.admin.shopee.io/api/v2/order/get_order_detail?access_token=access_token&order_sn_list=201214JAJXU6G7%2C201214JASXYXY6&partner_id=partner_id&request_order_status_pending=true&response_optional_fields=total_amount&shop_id=shop_id&sign=sign&timestamp=timestamp"

headers = {
    "Content-Type": "application/json"  # Example header if needed, modify or add other headers as required
}

response = requests.get(url, headers=headers)

print(response.text)


ConnectTimeout: HTTPSConnectionPool(host='open.admin.shopee.io', port=443): Max retries exceeded with url: /api/v2/order/get_order_detail?access_token=access_token&order_sn_list=201214JAJXU6G7%2C201214JASXYXY6&partner_id=partner_id&request_order_status_pending=true&response_optional_fields=total_amount&shop_id=shop_id&sign=sign&timestamp=timestamp (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019868C81B50>, 'Connection to open.admin.shopee.io timed out. (connect timeout=None)'))

In [9]:
import tkinter
print("Tkinter is available")

Tkinter is available


In [ ]:
# Konstanta
OUTPUT_FOLDER = "output"
BASE_URL = 'https://shopee.co.id/api/v4/recommend/recommend?bundle=shop_page_product_tab_main&limit=999&offset=0&section=shop_page_product_tab_main_sec&shopid='
HEADERS = ["ad_id", "judul", "stok", "harga", "penjualan", "rating", "suka", "tampilan", "deskripsi"]

# Memeriksa dan membuat folder output jika diperlukan
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Inisialisasi executor global
executor = ThreadPoolExecutor(max_workers=20)

# Fungsi untuk mengambil deskripsi produk dengan Selenium dan BeautifulSoup
def fetch_product_description_with_selenium(item_id):
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")

        # Menambahkan User-Agent
        chrome_options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, seperti Gecko) Chrome/58.0.3029.110 Safari/537")

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
        driver.get(f"https://shopee.co.id/product/{item_id}/")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)  # Menunggu halaman selesai dimuat
        wait = WebDriverWait(driver, 10)
        description_element = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.shopee-product-detail__description')))

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        description_element = soup.select_one(
            ".shopee-product-detail__description")  # CSS Selector untuk menemukan elemen deskripsi
        description = description_element.text if description_element else "Deskripsi tidak ditemukan"

        driver.quit()
        return description, None
    except Exception as e:
        return None, f"Kesalahan dalam ekstraksi menggunakan Selenium dan BeautifulSoup: {e}"

# Fungsi untuk mengambil data Shopee
def fetch_shopee_data(seller_id):
    try:
        response = requests.get(BASE_URL + seller_id, timeout=5)
        response.raise_for_status()
        return response.json(), None
    except requests.RequestException as e:
        return None, f"Kesalahan dalam permintaan: {e}"

# Fungsi untuk mengekstrak seller_id dari link
def extract_seller_id(link):
    match = re.search(r'i\.(\d+)\.', link)
    if match:
        return match.group(1), None
    return None, "Link tidak valid atau seller_id tidak ditemukan"

# Fungsi untuk menyimpan informasi ke CSV
def save_to_csv(data, path):
    with open(path, "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)

# Fungsi untuk mengunduh gambar
def download_images(images, folder):
    for idx, image in enumerate(images):
        image_url = f"https://cf.shopee.co.id/file/{image}_tn"
        urllib.request.urlretrieve(image_url, f"{folder}/image_{idx}.jpg")

# Fungsi untuk memulai scraping
def start_scraping():
    input_data = clientid_entry.get().strip()
    log_text.delete(1.0, tk.END)
    progress_bar["value"] = 0

    if not input_data:
        messagebox.showerror("Kesalahan", "Silakan masukkan clientid atau tautan produk yang valid.")
        return

    if "shopee" in input_data:
        seller_id, error = extract_seller_id(input_data)
        if error:
            messagebox.showerror("Kesalahan", error)
            return
    else:
        seller_id = input_data

    shopee_data, error = fetch_shopee_data(seller_id)
    if error:
        messagebox.showerror("Kesalahan", error)
        return

    total_products = len(shopee_data['data']['sections'][0]['data']['item'])
    remaining_label.config(text=f"Total produk: {total_products}")
    start_button.config(state=tk.NORMAL)

# Fungsi untuk melakukan scraping
def perform_scraping():
    global executor
    executor = ThreadPoolExecutor(max_workers=20)
    input_data = clientid_entry.get().strip()
    start_time = time.time()

    if "shopee" in input_data:
        seller_id, _ = extract_seller_id(input_data)
    else:
        seller_id = input_data

    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    seller_folder = f"{OUTPUT_FOLDER}/{seller_id}_{current_time}"
    if os.path.exists(seller_folder):
        messagebox.showinfo("Info", "Scraping sudah dilakukan untuk clientid ini.")
        return

    os.makedirs(seller_folder, exist_ok=True)
    csv_path = f"{seller_folder}/informasi_{current_time}.csv"
    save_to_csv(HEADERS, csv_path)

    shopee_data, _ = fetch_shopee_data(seller_id)
    total_products = len(shopee_data['data']['sections'][0]['data']['item'])

    for i, ad in enumerate(shopee_data['data']['sections'][0]['data']['item']):
        executor.submit(save_product_data, ad, i, total_products, seller_folder, csv_path)

    end_time = time.time()
    elapsed_time = end_time - start_time
    status_label.config(text=f"Scraping selesai dalam {elapsed_time:.2f} detik.")

# Fungsi untuk menyimpan data produk
def save_product_data(ad, index, total, seller_folder, csv_path):
    start_time = time.time()
    ad_id = ad['itemid']
    title = ad['name']
    log_text.insert(tk.END, f"Menyimpan produk: {title}\n")
    log_text.yview(tk.END)
    remaining_label.config(text=f"Produk tersisa: {total - index - 1}")

    # Mengambil deskripsi produk menggunakan Selenium
    description, error = fetch_product_description_with_selenium(ad_id)
    if error:
        description = "Tidak dapat memperoleh deskripsi"

    # Menyimpan data ke CSV
    save_to_csv([ad_id, title, ad['stock'], ad['price'], ad['historical_sold'], ad['item_rating']['rating_count'][0],
                 ad['liked_count'], ad['view_count'], description], csv_path)

    ad_folder = f"{seller_folder}/{ad_id}"
    os.makedirs(ad_folder, exist_ok=True)
    download_images(ad['images'], ad_folder)

    # Menyimpan deskripsi ke file .txt
    with open(f"{ad_folder}/description.txt", "w", encoding='utf-8') as f:
        f.write(description)

    progress_bar["value"] = (index + 1) / total * 100

    end_time = time.time()
    elapsed_time = end_time - start_time
    log_text.insert(tk.END, f"Produk: {title} disimpan dalam {elapsed_time:.2f} detik.\n")

# Antarmuka Grafis
root = tk.Tk()
root.title("Shopee Scraper Final")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

clientid_label = ttk.Label(frame, text="Masukkan clientid atau tautan produk:")
clientid_label.grid(row=0, column=0, sticky=tk.W, pady=5)
clientid_entry = ttk.Entry(frame, width=40)
clientid_entry.grid(row=0, column=1, sticky=tk.W, pady=5)

fetch_button = ttk.Button(frame, text="Cari Data", command=start_scraping)
fetch_button.grid(row=1, columnspan=2, pady=5)

start_button = ttk.Button(frame, text="Mulai Scraping", state=tk.DISABLED, command=perform_scraping)
start_button.grid(row=2, columnspan=2, pady=5)

progress_bar = ttk.Progressbar(frame, orient="horizontal", length=300, mode="determinate")
progress_bar.grid(row=3, columnspan=2, pady=5)

status_label = ttk.Label(frame, text="Status: Menunggu")
status_label.grid(row=4, columnspan=2, sticky=tk.W, pady=5)

remaining_label = ttk.Label(frame, text="Total produk: N/A")
remaining_label.grid(row=5, columnspan=2, sticky=tk.W, pady=5)

log_text = scrolledtext.ScrolledText(frame, width=50, height=10)
log_text.grid(row=6, columnspan=2, pady=10)

root.mainloop()
